# Data Storage and Access Patterns (including Astra Vector Search)



# Table Schema

```
CREATE TABLE IF NOT EXISTS workspan.customer_opportunities (
    customer_id text,
    partner_id text,
    opportunity_id text,
    customer_name text static,
    llm_output text,
    opportunity map<text, text>,
    llm_output_embedding vector<float, 1536>,
    sentiment text,
    PRIMARY KEY ((customer_id, partner_id), opportunity_id)
) WITH CLUSTERING ORDER BY (opportunity_id DESC)

```



* customer_id: Unique identifier for each customer
* partner_id: AWS / Azure / GCP
* opportunity_id: Unique identifier for each opportunity
* opportunity: Dynamic collection of data field names and corresponindg values specific to the opportunity
* llm_ouput: LLM output summarizing the 'next steps' , 'challenges' and 'open items'  
* llm_ouput_embedding: Text embeddings corresponding to llm_output
* sentiment: Positive, Neutral and Negative sentiment derived from 'next step' and 'cadence' data fields. This can be detrmined either by using a sentiment analysis library such as  Python Natural Language Toolkit (NLTK) or from LLM.


# Imports

In [ ]:
!pip install openai cassandra-driver llama-index

In [9]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import openai
from llama_index import ListIndex
from llama_index.readers.schema.base import Document
from IPython.display import Markdown, display

# Keys & Environment Variables

In [3]:
# keys and tokens here
openai_api_key = "<open_api_key>"
openai.api_key = openai_api_key
cass_user = '<user>'
cass_pw = '<pwd>'
scb_path = 'secure-connect-vector-search-demo.zip'

# Select a model to compute embeddings

In [4]:
model_id = "text-embedding-ada-002"

# Connect to the Cluster

In [5]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace('workspan')

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(138087894422944) 78e5fc41-a92d-43ee-be98-1dc613a9792d-us-east1.db.astra.datastax.com:29042:2a7c8292-de7b-44eb-8409-c5a3c3fc70fc> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


# Drop / Create Schema

In [42]:

# # Create Table
session.execute(f"""CREATE TABLE IF NOT EXISTS workspan.customer_opportunities (
    customer_id text,
    partner_id text,
    opportunity_id text,
    customer_name text static,
    llm_output text,
    opportunity map<text, text>,
    llm_output_embedding vector<float, 1536>,
    sentiment text,
    PRIMARY KEY ((customer_id, partner_id), opportunity_id)
) WITH CLUSTERING ORDER BY (opportunity_id DESC)""")



In [43]:
session.execute(f"""CREATE CUSTOM INDEX ON workspan.customer_opportunities(opportunity) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")

In [45]:
session.execute(f"""CREATE CUSTOM INDEX ON workspan.customer_opportunities (llm_output_embedding) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")

In [46]:
session.execute(f"""CREATE CUSTOM INDEX ON workspan.customer_opportunities (sentiment) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")

# Create the following SAI with analyser from CQL


```
CREATE CUSTOM INDEX ON workspan.customer_opportunities(llm_output) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = { 'index_analyzer': '{
"tokenizer" : { "name" : "ngram", "args" : {
"minGramSize":"2",
"maxGramSize":"3" }
},
"filters" : [
{
"name" : "lowercase", "args": {}
}
],
"charFilters" : []
}' };
```



# Calculate the sentiment

Sentiment can be calculated either by using a sentiment analysis library such as Python Natural Language Toolkit (NLTK) or from LLM.

In [47]:
def indentify_sentiment(next_step_and_cadence):
    import nltk
    from nltk.sentiment.vader import SentimentIntensityAnalyzer

    nltk.download('vader_lexicon')
    sid = SentimentIntensityAnalyzer()

    sentiment_scores = sid.polarity_scores(next_step_and_cadence)
    if sentiment_scores['compound'] >= 0.05:
        sentiment = 'positive'
    elif sentiment_scores['compound'] <= -0.05:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'

    print(f"Sentiment: {sentiment}")
    return sentiment


# Generate LLM ouput

In [48]:
def extract_llm_information(next_step_and_cadence):
    llm_input = f"""
                Given the following information, please list out the challenges, next steps, and open items mentioned. Do not list 'next steps' if there is no follow-up meeting that needs to be scheduled or if it's mentioned that no further follow-up is required:

                {next_step_and_cadence}

                End of information.
                """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Use the instructions provided to process the information."},
            {"role": "user", "content": llm_input}
        ]
    )

    return response.choices[0].message['content']

# Insert records with the calculated sentiment and LLM embeddings

In [50]:
query = SimpleStatement(
            f"""
            INSERT INTO workspan.customer_opportunities
            (customer_id, partner_id, opportunity_id, customer_name, llm_output,  opportunity , llm_output_embedding , sentiment )
            VALUES (%s, %s, %s, %s, %s , %s , %s , %s )
            """
        )

# record #1

next_step = f"""
Action Items:
From Michael, confirmed deprioritize. From Anjaney, account executive interest to schedule meeting - Anjaney to schedule call with Nirav/Amy on R&D.
"""

cadence = f"""
Next Step:
08/16/2023 : Review partner information updates and update opportunity details. 8/17(LR) - connecting with Partner to offer co-sell support

Next Step History:
null;08/16/2023 : Review partner information updates and update opportunity details.;08/16/2023 : Review partner information updates and update opportunity details. 8/17(LR) - connecting with Partner to offer co-sell support
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.Embedding.create(input= llm_output, model=model_id)['data'][0]['embedding']
session.execute(query, ('CUS100' , 'AWS', 'WS-7202838a', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))

# record #2

next_step = f"""
Action Items:
From Autumn, send recording of last call and our discussed inputs from demo 8/28. Ramesh will provide to Caroline by early next week (of 9/11).
"""

cadence = f"""
REVIEW TECH & Economic Proposal
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.Embedding.create(input= llm_output, model=model_id)['data'][0]['embedding']
session.execute(query, ('CUS100' , 'AWS', 'WS-8a038b8a', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))

# record #3

next_step = f"""
Action Items:
Joint sync set for 9/7. Enablement session to follow + in person account mapping. Caroline / Michael to begin coordinating. EAI presence
"""

cadence = f"""
07/05/2023: Contact Federico Gandolfo,federico.hernan.gandolfo@abc.com,+54.911.3204.4871 to discuss Deal support
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.Embedding.create(input= llm_output, model=model_id)['data'][0]['embedding']
session.execute(query, ('CUS100' , 'AWS', 'WS-8a3b0348', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))

# record #4

next_step = f"""
Action Items:
From Caroline, user community engaged to respond to questions. @Dataiku - How can we get initial data from user community/pull together PoV for client? Action (Asan/Ken (sp?)): In-person outreach to Deloitte users and follow-up to 5 responses received.
"""

cadence = f"""
null;06/20/2023: Contact Federico Gandolfo,federico.hernan.gandolfo@abc.com,+54.911.3204.4871 to discuss Deal support;07/05/2023: Contact Federico Gandolfo,federico.hernan.gandolfo@abc.com,+54.911.3204.4871 to discuss Deal support
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.Embedding.create(input= llm_output, model=model_id)['data'][0]['embedding']
session.execute(query, ('CUS100' , 'AWS', 'WS-8a7128a3', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))

# record #4

next_step = f"""
Propsal did not go thru. No budget Left. Negative.
"""

cadence = f"""
No further follow up required.
"""

next_step_and_cadence = next_step + cadence

sentiment = indentify_sentiment(next_step_and_cadence)
llm_output = extract_llm_information(next_step_and_cadence)
print(f"{llm_output}/n")
embedding_llm_output = openai.Embedding.create(input= llm_output, model=model_id)['data'][0]['embedding']
session.execute(query, ('CUS100' , 'AWS', 'WS-8a7128a4', 'Teradyne, Inc.', llm_output,  {'Customer State' : 'Ile-de-France', 'Customer Country' : 'France', 'Deal Size' : '30000', 'Description' : 'Pain Point: Persistent phishing attacks and email compromises leading to data leaks and compromised accounts. Description: The customer is struggling with the recurring threat of phishing attacks that infiltrate their systems, compromising sensitive data and risking the confidentiality of critical information.' }
                        , embedding_llm_output , sentiment))



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: positive
Challenges:
- The main challenge is the deprioritization of the project by Michael.

Next Steps:
- Anjaney needs to schedule a call with Nirav and Amy on R&D.
- Review partner information updates and update opportunity details on 08/16/2023.
- Connect with the partner on 08/17 to offer co-sell support.

Open Items:
- There are no specific open items mentioned in the provided information./n


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: neutral
Challenges:
- No challenges mentioned in the given information.

Next Steps:
- Autumn to send recording of last call and discussed inputs from demo on 8/28 to Caroline.
- Ramesh to provide the recording and discussed inputs to Caroline by early next week (of 9/11).

Open Items:
- No open items mentioned in the given information./n


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: positive
Challenges:
1. The coordination of the joint sync and enablement session needs to be managed by Caroline and Michael.
2. The presence of EAI needs to be ensured for the joint sync.

Next Steps:
1. Schedule a joint sync for 9/7 and coordinate the enablement session with Caroline and Michael.
2. Contact Federico Gandolfo at federico.hernan.gandolfo@abc.com or +54.911.3204.4871 to discuss deal support.

Open Items:
None mentioned in the given information./n


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: positive
Challenges mentioned:
- Engaging the user community to respond to questions
- Getting initial data from the user community and pulling together a Point of View (PoV) for the client
- In-person outreach to Deloitte users and follow-up to 5 responses received

Next Steps:
- In-person outreach to Deloitte users and follow-up on 5 responses received

Open Items:
- It is unclear if any further follow-up is required or if a follow-up meeting needs to be scheduled./n


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment: negative
Based on the given information, the challenges mentioned are:
1. The proposal did not go through.
2. There is no budget left.
3. The situation is negative.

The next steps are not mentioned in the provided information.

The open items mentioned are:
1. No further follow-up is required./n


# Opportunity Specific Queries

What is the customer sentiment on this opportunity?

In [57]:
cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = 'WS-7202838a'  ;'''
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    customer_id:      {row.customer_id}')
    print(f'    partner_id:      {row.partner_id}')
    print(f'    opportunity_id:      {row.opportunity_id}')
    print(f'    customer_name:      {row.customer_name}')
    print(f'    sentiment:      {row.sentiment}')

print('\n...')


Row 0:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-7202838a
    customer_name:      Teradyne, Inc.
    sentiment:      positive

...


What are the next steps for this opportunity? Result can be further parsed to display only the next steps. Same query returns open items and challenges as well.  

In [56]:
#embedding = openai.Embedding.create(input= vectorsearchon, model=model_id)['data'][0]['embedding']

cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = 'WS-7202838a';  '''
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    customer_id:      {row.customer_id}')
    print(f'    partner_id:      {row.partner_id}')
    print(f'    opportunity_id:      {row.opportunity_id}')
    print(f'    customer_name:      {row.customer_name}')
    print(f'    llm_output:    \n  {row.llm_output}')

print('\n...')


Row 0:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-7202838a
    customer_name:      Teradyne, Inc.
    llm_output:    
  Challenges:
- The main challenge is the deprioritization of the project by Michael.

Next Steps:
- Anjaney needs to schedule a call with Nirav and Amy on R&D.
- Review partner information updates and update opportunity details on 08/16/2023.
- Connect with the partner on 08/17 to offer co-sell support.

Open Items:
- There are no specific open items mentioned in the provided information.

...


# Customer Specific Queries (across multiple opportunities)

Identify the wins

In [55]:
cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and sentiment = 'positive'  ;'''
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    customer_id:      {row.customer_id}')
    print(f'    partner_id:      {row.partner_id}')
    print(f'    opportunity_id:      {row.opportunity_id}')
    print(f'    customer_name:      {row.customer_name}')
    print(f'    sentiment:      {row.sentiment}')

print('\n...')


Row 0:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-8a7128a3
    customer_name:      Teradyne, Inc.
    sentiment:      positive

Row 1:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-8a3b0348
    customer_name:      Teradyne, Inc.
    sentiment:      positive

Row 2:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-7202838a
    customer_name:      Teradyne, Inc.
    sentiment:      positive

...


Identify opportunities with next step to schedule a meeting

In [52]:
vectorsearchon = 'next action to set up a meeting'
embedding = openai.Embedding.create(input= vectorsearchon, model=model_id)['data'][0]['embedding']

cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and llm_output : 'schedule' ORDER BY llm_output_embedding ANN OF {embedding} LIMIT 10;  '''
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    customer_id:      {row.customer_id}')
    print(f'    partner_id:      {row.partner_id}')
    print(f'    opportunity_id:      {row.opportunity_id}')
    print(f'    customer_name:      {row.customer_name}')
    print(f'    sentiment:      {row.sentiment}')
    print(f'    llm_output:    \n  {row.llm_output}')

print('\n...')



Row 0:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-8a7128a3
    customer_name:      Teradyne, Inc.
    sentiment:      positive
    llm_output:    
  Challenges mentioned:
- Engaging the user community to respond to questions
- Getting initial data from the user community and pulling together a Point of View (PoV) for the client
- In-person outreach to Deloitte users and follow-up to 5 responses received

Next Steps:
- In-person outreach to Deloitte users and follow-up on 5 responses received

Open Items:
- It is unclear if any further follow-up is required or if a follow-up meeting needs to be scheduled.

Row 1:
    customer_id:      CUS100
    partner_id:      AWS
    opportunity_id:      WS-7202838a
    customer_name:      Teradyne, Inc.
    sentiment:      positive
    llm_output:    
  Challenges:
- The main challenge is the deprioritization of the project by Michael.

Next Steps:
- Anjaney needs to schedule a call with Nirav and Amy on R&D.


I want to know more about the customer and the challenges so that it is possible to plan accordingly (Implement query using agent framework such as LangChain, LlamaIndex)

In [54]:
vectorsearchon = 'Challenges'
embedding = openai.Embedding.create(input= vectorsearchon, model=model_id)['data'][0]['embedding']

cqlSelect = f'''SELECT llm_output FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' ORDER BY llm_output_embedding ANN OF {embedding} LIMIT 10;  '''
rows = session.execute(cqlSelect)
print(rows)
documents = []
for item in rows:
    documents.append(Document(text=str(item)))
    print(str(item))

index = ListIndex.from_documents(documents)

# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What are the Challenges?")

# visualize in console or web
print(response)
display(Markdown(f"<b>{response}</b>"))

Row(llm_output='Challenges mentioned:\n- Engaging the user community to respond to questions\n- Getting initial data from the user community and pulling together a Point of View (PoV) for the client\n- In-person outreach to Deloitte users and follow-up to 5 responses received\n\nNext Steps:\n- In-person outreach to Deloitte users and follow-up on 5 responses received\n\nOpen Items:\n- It is unclear if any further follow-up is required or if a follow-up meeting needs to be scheduled.')
Row(llm_output='Challenges:\n- No challenges mentioned in the given information.\n\nNext Steps:\n- Autumn to send recording of last call and discussed inputs from demo on 8/28 to Caroline.\n- Ramesh to provide the recording and discussed inputs to Caroline by early next week (of 9/11).\n\nOpen Items:\n- No open items mentioned in the given information.')
Row(llm_output='Based on the given information, the challenges mentioned are:\n1. The proposal did not go through.\n2. There is no budget left.\n3. The s

<b>The challenges mentioned in the given context information are as follows:
1. Engaging the user community to respond to questions
2. Getting initial data from the user community and pulling together a Point of View (PoV) for the client
3. In-person outreach to Deloitte users and follow-up to 5 responses received
4. The main challenge is the deprioritization of the project by Michael
5. The coordination of the joint sync and enablement session needs to be managed by Caroline and Michael
6. The presence of EAI needs to be ensured for the joint sync.</b>